<a href="https://colab.research.google.com/github/SrijaDasgupta/Digit-Recognizer/blob/main/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd/content/drive/MyDrive/digit-recognizer

/content/drive/MyDrive/digit-recognizer


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization,Dense,Conv2D,Input,MaxPooling2D,Dropout,Flatten,Add
from tensorflow.keras.optimizers import Adam

In [3]:
train = pd.read_csv('/content/drive/MyDrive/digit-recognizer/train.csv') # read training dataframe

In [4]:
X = train.iloc[:,1:].values  # set training data
y = train.iloc[:,0].values # set training labels
X = X.reshape(-1,28,28,1) # reshape into a format that can be fed into our NN
X = X / 255 # normalize our training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42) # split data into training and validation

In [5]:
inputs = Input((28,28,1))
              
x = Conv2D(filters = 32, kernel_size = 5, strides = 1, activation = 'relu' , kernel_regularizer=tf.keras.regularizers.l2(0.0005))(inputs)
x = Conv2D(filters = 32, kernel_size = 5, strides = 1, use_bias=False,activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(strides = 2)(x)
x = Dropout(0.3)(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, activation = 'relu', kernel_regularizer=l2(0.0005))(x)
x = Conv2D(filters = 64, kernel_size = 3, strides = 1, use_bias=False,activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(strides = 2)(x)
x = Dropout(0.3)(x)
    
    
x = Flatten()(x)
x = Dense(units = 256, use_bias=False,activation='relu')(x)
x = BatchNormalization()(x)

x = Dense(units = 128, use_bias=False,kernel_regularizer=l2(0.0005),activation='relu')(x)
x = BatchNormalization()(x)

x1 = Dense(units = 84, use_bias=False,kernel_regularizer=l2(0.0005),activation='relu')(x)
x = BatchNormalization()(x1)
x2 = Dropout(0.3)(x)
x = tf.keras.layers.Add()([x1,x2])

outputs = Dense(units = 10, activation = 'softmax')(x)
model1 = Model(inputs=inputs, outputs=outputs)

In [6]:
def scheduler(epoch, lr):
    
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler) # set the callback to our scheduler function

In [ ]:
model1.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer= tf.keras.optimizers.Adam(learning_rate=0.01))
r = model1.fit(X,y,epochs=50,callbacks=callback,validation_data=(X_valid, y_valid))

In [ ]:
import tensorflow as tf

class CustomLoss(Loss):

    def __init__(self)
        super.__init__()


    def call(y_true, y_pred):
        squared_difference = tf.square(y_true - y_pred)
        return tf.reduce_mean(squared_difference, axis=-1)

In [ ]:
class CustomDense(Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units


    def build(self, input_shape): # Creates the weights of the layer
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name='kernel',
                 initial_value = w_init(shape=(input_shape[-1], self.units), dtype='float32'),
                 trainable=True)

        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(self.units,), dtype='float32'),
        trainable=True)


    def call(self, inputs): # Defines the computation from the inputs to the outputs
         return tf.matmul(inputs, self.w) + self.b

In [ ]:
class DetectOverfittingCallback(tf.keras.callbacks.Callback):
    def __init__(self, units):
        super().__init__()
        self.threshold = threshold


    def on_epoch_end(self, epoch, log=None): # Creates the weights of the layer
        ratio = logs['val_loss'] / logs['loss']
        print(f'Epoch: {epoch}, Val/Train loss ratio: {ratio}')

        if ratio>threshold:
            print('Training stopped')
            self.model.stop_training = True

In [ ]:
import tensorflow as tf

class MyModel(tf.keras.Model):

    def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(16, activation='relu',input_shape=(784,))
    self.dense2 = tf.keras.layers.Dense(32, activation='softmax')

    def call(self, inputs, training=False):
    x = self.dense1(inputs,training=training)
    return self.dense2(x)

model = MyModel()

In [ ]:
class ConvBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size):
        super().__init__()

        self.conv1 = Conv2D(filters, kernel_size, padding='same',activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.conv2 = Conv2D(filters, kernel_size, padding='same',activation='relu')
        self.batchnorm = BatchNormalization()
        self.maxpool = MaxPooling2D(strides = 2)
        self.dropout = Dropout(0.3)
    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.conv2(x)
        x = self.batchnorm(x)
        x = self.maxpool(x)
        x = self.dropout(x)
        return x

In [ ]:
class DenseBlock(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()

        self.dense1 = Dense(units = units, use_bias=False, kernel_regularizer=l2(0.0005),activation='relu')
        self.batchnorm = BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.dense1(input_tensor)
        x = self.batchnorm(x,training=training)
        return x

In [ ]:
class MyModel(tf.keras.Model):

    def __init__(self,num_classes=10):
        super().__init__()
        self.convblock1 = ConvBlock(32,5)
        self.convblock2 = ConvBlock(64,5)
        self.flatten = Flatten()
        self.denseblock1 = DenseBlock(256)
        self.denseblock2 = DenseBlock(128)
        self.denseblock3 = DenseBlock(84)
        self.dropout = Dropout(0.3)
        self.outputs = Dense(units = num_classes, activation = 'softmax')
    
    def call(self, input_tensor, training=False):
        x = self.convblock1(input_tensor)
        x = self.convblock2(x)
        
        x = self.flatten(x)
        x = self.denseblock1(x)
        x = self.denseblock2(x)
        x = self.denseblock3(x)
        x = self.dropout(x)
        x = self.outputs(x)
        return x

In [ ]:
model2 = MyModel(num_classes=10)
model2.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer= tf.keras.optimizers.Adam(learning_rate=0.01))

In [ ]:
model2.fit(X,y,epochs=50,callbacks=callback,validation_data=(X_valid, y_valid))